# Ticket System

In [1]:
import instructor
from pydantic import BaseModel, Field
from enum import Enum

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Ticket System: Structured Output

In [2]:
# Patch the OpenAI client
client = instructor.from_openai(OpenAI())


class TicketCategory(str, Enum):
    """수신 티켓의 카테고리 열거하기"""

    GENERAL = "general"
    ORDER = "order"
    BILLING = "billing"


class CustomerSentiment(str, Enum):
    """고객 감정 레이블의 열거하기"""

    NEGATIVE = "negative"
    NEUTRAL = "neutral"
    POSITIVE = "positive"


class Ticket(BaseModel):
    reply: str = Field(description="Your reply that we send to the customer.")
    category: TicketCategory
    confidence: float = Field(ge=0, le=1)
    sentiment: CustomerSentiment


def process_ticket(customer_message: str) -> Ticket:
    reply = client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Ticket,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": "수신되는 고객 메시지를 분석하고 티켓의 값을 예측하세요.",
            },
            {"role": "user", "content": customer_message},
        ],
    )

    return reply

## Billing Issue Example

In [3]:
ticket = process_ticket("안녕하세요, 청구서에 대한 질문이 있습니다. 도와주실 수 있나요?")
assert ticket.category == TicketCategory.BILLING

ticket.reply
ticket.category
ticket.confidence
ticket.sentiment

<CustomerSentiment.NEUTRAL: 'neutral'>

## Order-Related Example

In [5]:
ticket = process_ticket("주문하고 싶습니다.")

assert ticket.category == TicketCategory.ORDER

ticket.reply
ticket.category
ticket.confidence
ticket.sentiment

<CustomerSentiment.POSITIVE: 'positive'>

In [6]:
ticket = process_ticket("물건에 하자가 있어 반품하고 싶습니다.")

assert ticket.category == TicketCategory.ORDER

ticket.reply
ticket.category
ticket.confidence
ticket.sentiment

<CustomerSentiment.NEGATIVE: 'negative'>